In [ ]:
import socket
import random
import time
import torch
import numpy as np
from Res import *
from MCTS_atax import *
from Connect_Ataxx import Atax as Connect_to_atax 
from atax import *

In [ ]:
# Escolha do board para o jogo
Games = ["A4x4", "A5x5", "A6x6"]
number = int(input("Escolha o jogo: 1-A4x4, 2-A5x5, 3-A6x6 "))
Ga = Games[number-1]

# Função para obter o tamanho do board
def n_board(Game):
    n = int(Game[1])
    return n

# Configuração do board
n_board = n_board(Ga)
definir_NB(n_board)
# Inicia o game para usar no mcts
game = Connect_to_atax(n_board)

# Escolha do device 
device = torch.device("cpu")

player = 1

# Argumentos para o mcts
args = {
    'C': 2,
    'num_searches': 100,
    'num_iterations': 1,
    'num_selfPlay_iterations': 20,
    'num_parallel_games': 10,
    'num_epochs': 10,
    'batch_size': 128,
    'temperature': 1.25,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3
}

# Inicia o modelo
model = ResNet(game, 9, 128, device)

# Load ao modelo já treinado
model.load_state_dict(torch.load("model_0_Atax.pt", map_location=torch.device('cpu')))
model.eval()

# Inicia o mcts
mcts = MCTS(game, args, model)

# Função para gerar o move do agente
def generate_move_from_model(state, ag):
    #Verifica se é o agente 1 ou 2
    if ag ==1:
        player = 1
    else:
        player = -1

    # Muda a perspectiva do tabuleiro para usar no mcts
    neutral_state = game.change_perspective(state, player)

    # Usa o mcts para obter as probabilidades
    mtcs_probs = mcts.search(neutral_state)

    # Escolhe a ação com maior probabilidade
    action = np.argmax(mtcs_probs)

    val = action//24
    xi= val//game.column_count
    yi=val%game.column_count
    pos= game.num_to_pos[action%24]
    xf = xi+pos[0]
    yf = yi+pos[1]
    
    # Dada a ação, retorna a jogada no formato "Xi,Yi,Xf,Yf"
    return f"MOVE {xi},{yi},{xf},{yf}"

# Função para extrair a jogada do agente adversário
def extract_row_col(move_str):
    row, col, row2, col2 = map(int, move_str.split(' ')[1].split(','))
    return row, col, row2, col2

# Função para se conectar ao servidor
def connect_to_server(host='localhost', port=12345):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    response = client_socket.recv(1024).decode()
    print(f"Server ResponseINIT: {response}")
    
    Game = response[-4:]
    print("Playing:", Game)
    
    atax = Connect_to_atax(n_board)
    initial_board = atax.get_initial_state() # tabuleiro inicial
    ata= State(initial_board,1) # jogo iniciado
    
    # Verifica se o server lhe atribuiu o agente 1 ou 2
    if "1" in response:
        ag=1
        player = 1
    else:
        ag=2
        player = -1
    
    # Inicia os estados do jogo
    state = ata.matrix
    first=True
    
    inv = 0 # Contador de jogadas inválidas
    
    # Loop do jogo
    while True:
        # Se é a vez do agente 1, gera a jogada e envia para o server
        if ag == 1 or not first:
            move = generate_move_from_model(state,ag)
            time.sleep(1)
            client_socket.send(move.encode())
            print("Send:", move)

            # Espera pela resposta do server
            response = client_socket.recv(1024).decode()
            print(f"Server Response1: {response}")
            if "END" in response:
                break
            if "VALID" == response:
                row, col, row2, col2 = extract_row_col(move)
                move_aux = Move(row, col, row2, col2,player, 0)
                move_aux.ty= move_aux.movement_type()
                ata.matrix =ata.execute_move(move_aux)
                print(ata.matrix)
            # Se a jogada for inválida, tenta enviar outra jogada
            elif "INVALID" in response:
                while "VALID" != response:
                    move = generate_move_from_model(state)
                    time.sleep(1)
                    client_socket.send(move.encode())
                    print("Send:", move)
                    response = client_socket.recv(1024).decode()
                    inv += 1
                    print(f"Server Response1: {response}")
                    # Se o contador de jogadas inválidas chegar a 3, o agente passa a vez
                    if inv == 3:
                        break
                # Se a jogada for válida, executa a jogada
                if "VALID" == response:
                    row, col, row2, col2 = extract_row_col(move)
                    move_aux = Move(row, col, row2, col2,player ,0)
                    move_aux.ty= move_aux.movement_type()
                    ata.matrix =ata.execute_move(move_aux)
                    print(ata.matrix)
                if "TURN LOSS" == response:
                    print(ata.matrix)
                    continue
            elif "TURN LOSS" == response:
                print(ata.matrix)
                continue
        
        pl = -player
        print("Player:", pl)
        first = False

        # Espera pela resposta do server, que é a jogada do agente adversário
        response = client_socket.recv(1024).decode()
        print(f"Server Response2: {response}")

        # Executa a jogada do agente adversário
        row, col, row2, col2 = extract_row_col(response)
        print(row, col, row2, col2)
        move_aux = Move(row, col, row2, col2,pl ,0)
        move_aux.ty= move_aux.movement_type()
        print(move_aux.ty)
        ata.matrix =ata.execute_move(move_aux)
        print(ata.matrix)
        
        state = ata.matrix
        # Se o jogo acabar, sai do ciclo
        if "END" in response:
            break

    # Fecha a conexão ao servidor
    client_socket.close()

if __name__ == "__main__":
    connect_to_server()